In [ ]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null  
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz  
!pip install -q findspark
!pip install isbnlib

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [ ]:
import findspark
findspark.init("spark-3.3.1-bin-hadoop3")
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql import Row
import pyspark.sql.functions as f
from pyspark.sql.functions import format_number
from pyspark.sql.functions import col, split, when
import pandas as pd
spark = SparkSession.builder.appName("Basic").getOrCreate()
import requests as req
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import ltrim
from pyspark.sql.functions import *
from isbnlib import is_isbn10, is_isbn13
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
from pyspark.sql.functions import trim
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Rating = spark.read.option("header", True).csv('/content/drive/MyDrive/Books_Rating/DataBase/BX-Book-Ratings.csv')
Rating.show(truncate = False)

+----------------------------------+----+----+
|"User-ID;""ISBN"";""Book-Rating"""|_c1 |_c2 |
+----------------------------------+----+----+
|"276725;""034545104X"";""0"""     |null|null|
|"276726;""0155061224"";""5"""     |null|null|
|"276727;""0446520802"";""0"""     |null|null|
|"276729;""052165615X"";""3"""     |null|null|
|"276729;""0521795028"";""6"""     |null|null|
|"276733;""2080674722"";""0"""     |null|null|
|"276736;""3257224281"";""8"""     |null|null|
|"276737;""0600570967"";""6"""     |null|null|
|"276744;""038550120X"";""7"""     |null|null|
|"276745;""342310538"";""10"""     |null|null|
|"276746;""0425115801"";""0"""     |null|null|
|"276746;""0449006522"";""0"""     |null|null|
|"276746;""0553561618"";""0"""     |null|null|
|"276746;""055356451X"";""0"""     |null|null|
|"276746;""0786013990"";""0"""     |null|null|
|"276746;""0786014512"";""0"""     |null|null|
|"276747;""0060517794"";""9"""     |null|null|
|"276747;""0451192001"";""0"""     |null|null|
|"276747;""06

In [ ]:
Rating = Rating.drop("_c1", "_c2").withColumnRenamed('"User-ID;""ISBN"";""Book-Rating"""', "User-ID").withColumn('User-ID', regexp_replace('User-ID', '"', ''))
Rating = Rating.withColumn('UserID', split(Rating['User-ID'], ';').getItem(0)) \
       .withColumn('ISBN', split(Rating['User-ID'], ';').getItem(1)) \
       .withColumn('Rating', split(Rating['User-ID'], ';').getItem(2))
Rating = Rating.drop("User-ID")

In [ ]:
Rating = Rating.withColumn("UserID",ltrim(col("UserID"))).withColumn("Rating",ltrim(col("Rating"))).withColumn("ISBN",ltrim(col("ISBN"))) \
        .withColumn("ISBN",trim(col("ISBN"))).withColumn("UserID",trim(col("UserID"))).withColumn("Rating",trim(col("Rating")))

In [ ]:
ValidISBN = []
InvalidISBN = []

row_listBOOKRATINGS = Rating.collect()

for i in row_listBOOKRATINGS:
  if (is_isbn10(i.__getitem__('ISBN')) == True):
    ValidISBN.append(i)
  else:
    InvalidISBN.append(i) 

InvalidISBN_ONLY = []
for i in InvalidISBN:
  InvalidISBN_ONLY.append(i.__getitem__('ISBN')) 

Rating = Rating.filter(~col('ISBN').isin(InvalidISBN_ONLY))

Rating = Rating.withColumnRenamed('UserID', 'UserBK').withColumnRenamed('ISBN', 'BookBK')

In [ ]:
Rating = Rating.withColumn("RatingSK", row_number().over(Window.orderBy(monotonically_increasing_id()))+100)
FACT_RATING = Rating.select('RatingSK', 'UserBK', 'BookBK', 'Rating')

FACT_RATING.show(truncate=False)
FACT_RATING.describe().show(truncate=False)

+--------+------+----------+------+
|RatingSK|UserBK|BookBK    |Rating|
+--------+------+----------+------+
|101     |276725|034545104X|0     |
|102     |276726|0155061224|5     |
|103     |276727|0446520802|0     |
|104     |276729|052165615X|3     |
|105     |276729|0521795028|6     |
|106     |276733|2080674722|0     |
|107     |276736|3257224281|8     |
|108     |276737|0600570967|6     |
|109     |276744|038550120X|7     |
|110     |276746|0425115801|0     |
|111     |276746|0449006522|0     |
|112     |276746|0553561618|0     |
|113     |276746|055356451X|0     |
|114     |276746|0786013990|0     |
|115     |276746|0786014512|0     |
|116     |276747|0060517794|9     |
|117     |276747|0451192001|0     |
|118     |276747|0609801279|0     |
|119     |276747|0671537458|9     |
|120     |276747|0679776818|8     |
+--------+------+----------+------+
only showing top 20 rows

+-------+-----------------+------------------+--------------------+------------------+
|summary|RatingSK      

In [ ]:
FACT_RATING.createTempView("Rating")

FACT_RATING = spark.sql(
    '''
    select * from Rating
    where length(BookBK) = 10
    '''
)

In [ ]:
FACT_RATING.createTempView("BooksRating")

spark.sql(
    '''
    select * from BooksRating
    where length(BookBK) != 10
    '''
).show()

+--------+------+------+------+
|RatingSK|UserBK|BookBK|Rating|
+--------+------+------+------+
+--------+------+------+------+



In [ ]:
pandasDF = FACT_RATING.toPandas()
pandasDF.to_csv(r'/content/drive/MyDrive/Books_Rating/Fixed_CSV/FACT_RATING.csv',mode='w+', encoding='utf-8', index=False)

In [ ]:
spark.stop()